In [1]:
import pandas as pd
import numpy as np

from memory_profiler import profile

from Pyfhel import Pyfhel, PyPtxt, PyCtxt

import torch
import torch.nn as nn

import time
import os
import sys

working_directory = "/home/falcetta/PINPOINT_Secret"

device = "cpu"
module_path = os.path.abspath(working_directory)
sys.path.append(module_path) 

from pycrcnn.net_builder.encoded_net_builder_ts import build_from_pytorch
from pycrcnn.crypto.crypto import encrypt_matrix, decrypt_matrix
from train_utils import *

from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_squared_error, mean_absolute_error

# Models

In [2]:
class Square(torch.nn.Module):
    def __init__(self):
        super().__init__()
 
    def forward(self, t):
        return torch.pow(t, 2)

class Cube(torch.nn.Module):
    def __init__(self):
        super().__init__()
 
    def forward(self, t):
        return torch.pow(t, 3)
    
class Printer(torch.nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, t):
        # print(t)
        print(t.shape)
        return t


class PINPOINT_1CONV(nn.Module):
    def __init__(self, input_size, output_horizon):
        super(PINPOINT_1CONV, self).__init__()

        n_kernels_1 = 32
        kernel_size_1 = 3
        out_conv_1 = n_kernels_1 * (input_size - kernel_size_1 + 1)

        self.main = nn.Sequential(           
            nn.Conv1d(in_channels=1, out_channels=n_kernels_1, kernel_size=kernel_size_1),
            Square(),
            nn.Flatten(),      
            
            nn.Linear(out_conv_1, int(out_conv_1/2)), #use without avgpool
            # nn.Linear(int(out_conv_1/2), output_horizon)   
            nn.Linear(int(out_conv_1/2), int(out_conv_1/4)),
            nn.Linear(int(out_conv_1/4), output_horizon)   
        )

    def forward(self, x):
        out = self.main(x)
        return out
    
    def __str__(self):
        return "PINPOINT_1CONV"

    
class PINPOINT_2CONV(nn.Module):
    def __init__(self, input_size, output_horizon):
        super(PINPOINT_2CONV, self).__init__()
        
        n_kernels_1 = 16
        n_kernels_2 = 32
        kernel_size_1 = 5
        kernel_size_2 = 3
        
        out_conv_1 = input_size - kernel_size_1 + 1
        out_conv_2 = n_kernels_2 * (out_conv_1 - kernel_size_2 + 1)

        self.main = nn.Sequential(           
            nn.Conv1d(in_channels=1, out_channels=n_kernels_1, kernel_size=kernel_size_1),
            Square(),
            nn.Conv1d(in_channels=n_kernels_1, out_channels=n_kernels_2, kernel_size=kernel_size_2),
            Square(),
            nn.Flatten(),      
            
            nn.Linear(out_conv_2, int(out_conv_2/2)), #use without avgpool
            # nn.Linear(int(out_conv_2/4), output_horizon)   
            nn.Linear(int(out_conv_2/2), int(out_conv_2/4)),
            nn.Linear(int(out_conv_2/4), output_horizon)   
        )

    def forward(self, x):
        out = self.main(x)
        return out
    
    def __str__(self):
        return "PINPOINT_2CONV"

In [3]:
N_EXPERIMENTS = 10
experiment_name = "AirlinePassengers"
seq_length = 12
forecast_horizon = 6
model_class = "PINPOINT_1CONV"

In [4]:
model = torch.load(f"{working_directory}/Experiments/models/{experiment_name}_{forecast_horizon}_{model_class}.pt")

In [5]:
model

PINPOINT_1CONV(
  (main): Sequential(
    (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,))
    (1): Square()
    (2): Flatten(start_dim=1, end_dim=-1)
    (3): Linear(in_features=320, out_features=160, bias=True)
    (4): Linear(in_features=160, out_features=80, bias=True)
    (5): Linear(in_features=80, out_features=6, bias=True)
  )
)

In [6]:
model_input = np.array([[0.5] for _ in range(0, 12)]).reshape(1, 1, 12)

In [7]:
times_plain = np.array([])
model.eval()

torch_input = torch.FloatTensor(model_input).cuda()

for _ in range(0, N_EXPERIMENTS):
    t0 = time.time()
    with torch.set_grad_enabled(False):
        expected_output = model(torch_input)

    times_plain = np.append(times_plain, time.time()-t0)

In [8]:
times_encrypted = np.array([])
expected_output = expected_output.cpu()

for i in range(0, N_EXPERIMENTS):

    HE = Pyfhel()    
    HE.contextGen(p=95337867, m=4096, intDigits=32, fracDigits=64) 
    HE.keyGen()
    HE.relinKeyGen(30, 3)

    encoded_model = build_from_pytorch(HE, model.cpu().main)
    encrypted_input = encrypt_matrix(HE, model_input)

    t0 = time.time()

    for layer in encoded_model:
        encrypted_input = layer(encrypted_input)

    times_encrypted = np.append(times_encrypted, time.time()-t0)

    result = decrypt_matrix(HE, encrypted_input)

    assert np.allclose(expected_output.numpy(), result,
                       rtol=1e-02, atol=1e-04)

In [9]:
print(f"Mean time requested over {N_EXPERIMENTS} (plain processing): {np.mean(times_plain):.2f}")
print(f"Mean time requested over {N_EXPERIMENTS} (encrypted processing): {np.mean(times_encrypted):.2f}")

Mean time requested over 10 (plain processing): 0.00
Mean time requested over 10 (encrypted processing): 45.06
